In [1]:
"""
Created on Mon Jul 04 14:17 2022

Prepare csv for chunks in time 

Author: @claraburgard
"""

'\nCreated on Mon Jul 04 14:17 2022\n\nPrepare csv for chunks in time \n\nAuthor: @claraburgard\n'

FOR EACH POINT:
- T and S extrapolated to ice draft depth
- Mean std T and S
- Distance to front
- Distance to the grounding line
- ice draft zonal and meridional slope in x- and y-direction
- bedrock zonal and meridional slope in x- and y-direction
- Ice draft depth
- Bathymetry
- utide
- Ice draft concentration
- Max bathymetry 
- Target: melt m ice per yr

In [2]:
import numpy as np
import xarray as xr
import os.path
from tqdm.notebook import trange, tqdm
import nn_functions.data_formatting as dfmt

In [3]:
%matplotlib inline

READ IN DATA

In [4]:
nemo_run0 = 'OPM018' #'OPM016','OPM018','OPM021'
nemo_run = nemo_run0

In [5]:
inputpath_data_clara='../../../../../burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask_clara = '../../../../../burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
inputpath_profiles = '../../../../../burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
inputpath_plumes = '../../../../../burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
inputpath_boxes = '../../../../../burgardc/SCRIPTS/basal_melt_param/data/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_melt = '../../../../../burgardc/SCRIPTS/basal_melt_param/data/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
outputpath_nn = '../../../../../burgardc/DATA/NN_PARAM/interim/INPUT_DATA/EXTRAPOLATED_ISFDRAFT_CHUNKS/'
inputpath_tides = '../../../../../burgardc/DATA/BASAL_MELT_PARAM/interim/TIDES/'

In [31]:
inputpath_data='../../data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = '../../data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
inputpath_profiles = '../../data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
inputpath_plumes = '../../data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
inputpath_boxes = '../../data/interim/BOXES/nemo_5km_'+nemo_run+'/'

inputpath_tides_clara = '../../../../../burgardc/DATA/BASAL_MELT_PARAM/interim/TIDES/'

#inputpath_melt = '../../../../../burgardc/SCRIPTS/basal_melt_param/data/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'

outputpath_melt_helen = '../../data/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
outputpath_nn_helen = '../../data/interim/INPUT_DATA/EXTRAPOLATED_ISFDRAFT_CHUNKS/'


Input variables

In [7]:
# T and S extrapolated to ice draft depth
T_S_2D_isfdraft = xr.open_mfdataset(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS.nc')#.sel(profile_domain=50)

In [8]:
# T and S mean and std
T_S_2D_meanstd = xr.open_dataset(inputpath_profiles + 'T_S_2D_meanstd_isf_draft_oneFRIS.nc')

In [9]:
# dIF, dGL
file_isf_orig = xr.open_dataset(inputpath_mask_clara+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

In [28]:
file_isf['Nisf']

<xarray.DataArray 'Nisf' (Nisf: 35)>
array([10, 11, 12, 13, 18, 22, 23, 24, 25, 30, 31, 33, 38, 39, 40, 42, 43, 44,
       45, 47, 48, 51, 52, 53, 54, 55, 58, 61, 65, 66, 69, 70, 71, 73, 75])
Coordinates:
  * Nisf     (Nisf) int64 10 11 12 13 18 22 23 24 25 ... 61 65 66 69 70 71 73 75
Attributes:
    standard_name:  ice shelf ID

In [10]:
map_lim = [-3000000,3000000]
file_mask_orig = xr.open_dataset(inputpath_data_clara+'other_mask_vars_Ant_stereo.nc')
file_mask_orig_cut = dfmt.cut_domain_stereo(file_mask_orig, map_lim, map_lim)
file_other = xr.open_dataset(inputpath_data_clara+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = dfmt.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data_clara+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = dfmt.cut_domain_stereo(file_conc, map_lim, map_lim)

In [11]:
# bathymetry, ice draft, concentration
file_bed_orig = file_mask_orig_cut['bathy_metry']
file_bed_corr = file_other_cut['corrected_isf_bathy']
file_draft = file_other_cut['corrected_isfdraft'] 
file_bed_goodGL = file_bed_orig.where(file_draft < file_bed_orig,file_bed_corr)
file_isf_conc = file_conc_cut['isfdraft_conc']

In [12]:
file_slope = xr.open_dataset(inputpath_mask+'nemo_5km_slope_info_bedrock_draft_latlon_oneFRIS.nc')

In [13]:
utide_file = xr.open_dataset(inputpath_tides_clara + 'tidal_velocity_nemo_Ant_stereo.nc').rename({'ttv':'u_tide'})
u_tide = dfmt.cut_domain_stereo(utide_file['u_tide'], map_lim, map_lim)

Target

In [14]:
outputpath_melt+'melt_rates_2D_NEMO.nc'

'../../../../../burgardc/SCRIPTS/basal_melt_param/data/processed/MELT_RATE/nemo_5km_OPM018/melt_rates_2D_NEMO.nc'

In [15]:
NEMO_melt_rates_2D = xr.open_mfdataset(outputpath_melt_helen+'melt_rates_2D_NEMO.nc')
melt_rate = NEMO_melt_rates_2D['melt_m_ice_per_y']

PREPARE ONE DATASET WITH EVERYTHING

In [16]:
geometry_2D = file_isf[['dGL', 'dIF']].merge(file_draft).merge(file_bed_goodGL).merge(file_slope).merge(file_isf_conc).merge(u_tide) 
geometry_2D['dIF'] = geometry_2D['dIF'].where(np.isfinite(geometry_2D['dIF']), np.nan)
time_dpdt_in = T_S_2D_isfdraft[['theta_in','salinity_in']].merge(T_S_2D_meanstd).merge(melt_rate)

In [17]:
geometry_2D_br, time_dpdt_in_br = xr.broadcast(geometry_2D,time_dpdt_in)

In [18]:
geometry_2D_br

<xarray.Dataset>
Dimensions:             (x: 1200, y: 1200, time: 29, profile_domain: 2)
Coordinates:
  * x                   (x) float64 -2.998e+06 -2.993e+06 ... 2.997e+06
  * y                   (y) float64 -2.997e+06 -2.992e+06 ... 2.998e+06
    latitude            (y, x) float64 -52.35 -52.38 -52.4 ... -52.38 -52.35
    Nisf                int64 75
    longitude           (y, x) float64 -135.0 -135.0 -135.1 ... 44.94 44.99
  * time                (time) int64 1980 1981 1982 1983 ... 2005 2006 2007 2008
  * profile_domain      (profile_domain) object 'close_cont_shelf' '1000'
Data variables:
    dGL                 (x, y, time, profile_domain) float64 nan nan ... nan nan
    dIF                 (x, y, time, profile_domain) float64 nan nan ... nan nan
    corrected_isfdraft  (x, y, time, profile_domain) float64 nan nan ... nan nan
    bathy_metry         (x, y, time, profile_domain) float64 3e+03 ... 3e+03
    slope_bed_lon       (x, y, time, profile_domain) float64 nan nan ... nan nan
    slope_bed_lat       (x, y, time, profile_domain) float64 nan nan ... nan nan
    slope_ice_lon       (x, y, time, profile_domain) float64 nan nan ... nan nan
    slope_ice_lat       (x, y, time, profile_domain) float64 nan nan ... nan nan
    isf_area            (x, y, time, profile_domain) float64 nan nan ... nan nan
    entry_depth_max     (x, y, time, profile_domain) float64 nan nan ... nan nan
    isfdraft_conc       (x, y, time, profile_domain) float64 0.0 0.0 ... 0.0 0.0
    u_tide              (x, y, time, profile_domain) float64 0.0 0.0 ... 0.0 0.0
Attributes:
    history:     Created with combine_mask_metadata() by C. Burgard. dGL, dIF...
    projection:  Polar Stereographic South (71S,0E)
    proj4:       +init=epsg:3031
    Note:        isf ID and individual isf characteristics can be found in ic...

In [19]:

time_dpdt_in_br

<xarray.Dataset>
Dimensions:           (x: 1200, y: 1200, time: 29, profile_domain: 2)
Coordinates:
  * x                 (x) float64 -2.998e+06 -2.993e+06 ... 2.992e+06 2.997e+06
  * y                 (y) float64 -2.997e+06 -2.992e+06 ... 2.993e+06 2.998e+06
  * time              (time) int64 1980 1981 1982 1983 ... 2005 2006 2007 2008
  * profile_domain    (profile_domain) object 'close_cont_shelf' '1000'
    longitude         (y, x) float64 -135.0 -135.0 -135.1 ... 44.89 44.94 44.99
    latitude          (y, x) float64 -52.35 -52.38 -52.4 ... -52.4 -52.38 -52.35
Data variables:
    theta_in          (x, y, time, profile_domain) float64 dask.array<chunksize=(1200, 1200, 29, 2), meta=np.ndarray>
    salinity_in       (x, y, time, profile_domain) float64 dask.array<chunksize=(1200, 1200, 29, 2), meta=np.ndarray>
    T_mean            (x, y, time, profile_domain) float64 nan nan ... nan nan
    S_mean            (x, y, time, profile_domain) float64 nan nan ... nan nan
    T_std             (x, y, time, profile_domain) float64 nan nan ... nan nan
    S_std             (x, y, time, profile_domain) float64 nan nan ... nan nan
    melt_m_ice_per_y  (x, y, time, profile_domain) float64 dask.array<chunksize=(1200, 1200, 29, 2), meta=np.ndarray>

In [20]:
final_input_xr = xr.merge([geometry_2D_br, time_dpdt_in_br]).sel(profile_domain='close_cont_shelf').transpose('y','x','time').drop('profile_domain').drop('Nisf').load()
                                                                                                                                    
                                                                                                                                    
                                                                                                                                    
                                                                                                                                    
                                                                                                                                    
                                                                                                                                    

In [21]:
final_input_xr

<xarray.Dataset>
Dimensions:             (y: 1200, x: 1200, time: 29)
Coordinates:
  * x                   (x) float64 -2.998e+06 -2.993e+06 ... 2.997e+06
  * y                   (y) float64 -2.997e+06 -2.992e+06 ... 2.998e+06
    latitude            (y, x) float64 -52.35 -52.38 -52.4 ... -52.38 -52.35
    longitude           (y, x) float64 -135.0 -135.0 -135.1 ... 44.94 44.99
  * time                (time) int64 1980 1981 1982 1983 ... 2005 2006 2007 2008
Data variables: (12/19)
    dGL                 (y, x, time) float64 nan nan nan nan ... nan nan nan nan
    dIF                 (y, x, time) float64 nan nan nan nan ... nan nan nan nan
    corrected_isfdraft  (y, x, time) float64 nan nan nan nan ... nan nan nan nan
    bathy_metry         (y, x, time) float64 3e+03 3e+03 3e+03 ... 3e+03 3e+03
    slope_bed_lon       (y, x, time) float64 nan nan nan nan ... nan nan nan nan
    slope_bed_lat       (y, x, time) float64 nan nan nan nan ... nan nan nan nan
    ...                  ...
    salinity_in         (y, x, time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    T_mean              (y, x, time) float64 nan nan nan nan ... nan nan nan nan
    S_mean              (y, x, time) float64 nan nan nan nan ... nan nan nan nan
    T_std               (y, x, time) float64 nan nan nan nan ... nan nan nan nan
    S_std               (y, x, time) float64 nan nan nan nan ... nan nan nan nan
    melt_m_ice_per_y    (y, x, time) float64 nan nan nan nan ... nan nan nan nan
Attributes:
    history:     Created with combine_mask_metadata() by C. Burgard. dGL, dIF...
    projection:  Polar Stereographic South (71S,0E)
    proj4:       +init=epsg:3031
    Note:        isf ID and individual isf characteristics can be found in ic...

In [22]:
final_input_xr_clean = final_input_xr.where(np.isfinite(final_input_xr['S_mean']))

In [33]:
# Save data for the NN

ds_kisf = final_input_xr_clean
df_kisf = ds_kisf.drop('longitude').drop('latitude').to_dataframe()
# remove rows where there are nans
clean_df_kisf = df_kisf.dropna()
clean_df_kisf = clean_df_kisf.where(clean_df_kisf['salinity_in']!=0).dropna()
clean_df_kisf.to_csv(outputpath_nn_helen + 'dataframe_input_isf.csv')

#final_input_xr_clean.to_csv(output_nn + 'dataframe_input_isf.csv')

PREPARE CLEAN DATAFRAME WITH ALL DATA TO SAVE AND FEED TO THE NN

In [ ]:
# CHECK WHERE PREVIOUS CHUNKS STOPPED
if os.path.exists(outputpath_nn_helen+'info_chunks.txt'):
    with open(outputpath_nn_helen+'info_chunks.txt') as f:
        lines = f.readlines()
    last_line = lines[-1]
    end_chunks = int(last_line.split(',')[0]) 
else:
    end_chunks = 0

In [ ]:
chunk_all = int(np.ceil(len(final_input_xr.time)/10))

for chunk_nb in range(chunk_all):
    
    if chunk_nb == (chunk_all - 1):
        end_step = len(final_input_xr.time)
    else:
        end_step = chunk_nb*10+10
        
    trange = final_input_xr_clean.time.isel(time=range(chunk_nb*10,end_step))
    print(trange.values)
    start_yy = trange[0].values
    end_yy = trange.max().values
    
    chunk_nb_new = chunk_nb+end_chunks+1
    file_info = open(outputpath_nn+'info_chunks.txt', 'a')
    file_info.write(str(chunk_nb_new).zfill(3)+','+nemo_run+','+str(start_yy)+','+str(end_yy)+'\n')
    file_info.close()

    for kisf in tqdm(file_isf.Nisf):
        ds_kisf = final_input_xr_clean.sel(time=trange).where(file_isf['ISF_mask'] == kisf, drop=True).drop('Nisf')

        df_kisf = ds_kisf.drop('longitude').drop('latitude').to_dataframe()
        # remove rows where there are nans
        clean_df_kisf = df_kisf.dropna()
        clean_df_kisf = clean_df_kisf.where(clean_df_kisf['salinity_in']!=0).dropna()
        clean_df_kisf.to_csv(outputpath_nn + 'dataframe_input_isf'+str(kisf.values).zfill(3)+'_'+str(chunk_nb_new).zfill(3)+'.csv')